In [9]:
from os import listdir
from os.path import isfile, join
mypath='enron_with_categories/1'
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f)) and f.endswith('.txt')]

In [13]:
!pip3 install mail-parser==3.15.0
!pip3 install --upgrade google-cloud-language==2.2.2

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [53]:
import mailparser
from google.cloud import language_v1

from google.cloud import storage
import datetime, json

In [55]:
client = storage.Client()
bucket = client.get_bucket('feature-store-mars21')

blob = bucket.get_blob('tmp/128802.txt')

mail = mailparser.parse_from_bytes(blob.download_as_bytes())
#print(f, mail.body)
print(mail.date)
#print(mail.mail_json)
mail = json.loads(mail.mail_json)
print(mail['from'][0][1])
to = [item[1] for item in mail['to']]
print(to)

2001-09-12 22:19:51
alan.comnes@enron.com
['alan.comnes@enron.com', 'ray.alvarez@enron.com', 'steve.walton@enron.com', 'susan.mara@enron.com', 'leslie.lawner@enron.com', 'w..cantrell@enron.com', 'donna.fulton@enron.com', 'jeff.dasovich@enron.com', 'l..nicolay@enron.com', 'd..steffes@enron.com', 'k..allen@enron.com', 'j..noske@enron.com', 'dave.perrino@enron.com', 'don.black@enron.com', 'robert.frank@enron.com', 'stephanie.miller@enron.com', 'barry.tycholiz@enron.com', 'sarah.novosel@enron.com', 'jennifer.thome@enron.com', '.hall@enron.com', 'susan.lindberg@enron.com']


In [30]:
client = language_v1.LanguageServiceClient()

# text_content = 'Grapes are good. Bananas are bad.'

# Available types: PLAIN_TEXT, HTML
type_ = language_v1.Document.Type.PLAIN_TEXT

# Optional. If not specified, the language is automatically detected.
# For list of supported languages:
# https://cloud.google.com/natural-language/docs/languages
language = "en"
document = {"content": mail['body'], "type_": type_, "language": language}

# Available values: NONE, UTF8, UTF16, UTF32
encoding_type = language_v1.EncodingType.UTF8

features = {
  "extract_syntax": False,
  "extract_entities": True,
  "extract_document_sentiment": True,
  "extract_entity_sentiment": False,
  "classify_text": True
}

features
# Returns overall sentiment of email. not sentence by sentence.
response = client.annotate_text(
    document= document, encoding_type= encoding_type, features=features )



In [51]:
# to get email sentiment
response.document_sentiment.magnitude


# list of entities
response.entities

# list of categories - receiving the most dominand
response.categories[0].name
response.categories[0].confidence




0.5400000214576721